### Code Build using 
- Anaconda Navigator
- Jupyter Notebook
- Python and python libaries 

### asumption 
- The spec.json file gives the details about the column names and the maximum number of characters each column can hold in the file.
- assuming there will be some data to work with, which we'll use to create a new file with the specified column widths. After that, we'll convert this file from a .txt format to a CSV format.

In [2]:
import json
import csv

In [7]:
# Load the spec.json file
with open('spec.json', 'r') as file:
    spec = json.load(file)

# Convert offsets to integers
offsets = list(map(int, spec["Offsets"]))

# Sample data to write into the fixed-width file
possible_sample_data = [
    ["John", "Doe", "M", "NY", "123 Elm St", "10001", "john.doe@example.com", "Software Engineer", "New York, NY", "2024-08-24"],
    ["Jane", "Smith", "F", "CA", "456 Oak St", "90001", "jane.smith@example.com", "Data Scientist", "Los Angeles, CA", "2024-08-23"],
    ["1234578000", "abcdeghijklabcd", "1234567788", "456668999", "1234567890123", "4567890", "1234567890", "abcdefghijklm", "nopqrstuvwxyz", "abcdefghijklm"],
    ["67890asdfasdf", "fghij123432asdfasdf", "4560098766", "7896666288823", "9876543210987", "0123456", "0987654321", "mnopqrstuvwxyz", "abcdefghijklm", "nopqrstuvwxyz"],
    # Add more rows as needed
]

# Function to parse data based on offset
def pad_data(value, length):
    return str(value).ljust(length)[:length]

# Generate fixed-width file content
fixed_width_lines = []

# Include header if specified
if spec["IncludeHeader"] == "True":
    header = ''.join([pad_data(col, offsets[i]) for i, col in enumerate(spec["ColumnNames"])])
    fixed_width_lines.append(header)

# Write data rows
for row in possible_sample_data:
    fixed_width_line = ''.join([pad_data(row[i], offsets[i]) for i in range(len(row))])
    fixed_width_lines.append(fixed_width_line)

# Write to a fixed-width file
fixed_width_file_path = 'fixed_width_file.txt'
with open(fixed_width_file_path, 'w', encoding=spec["FixedWidthEncoding"]) as f:
    f.write('\n'.join(fixed_width_lines))

In [8]:
# Function to parse a fixed-width line into fields
def parse_fixed_width_line(line, offsets):
    fields = []
    start = 0
    for offset in offsets:
        fields.append(line[start:start + offset].strip())
        start += offset
    return fields

# Parse the fixed-width file and write to a CSV
csv_file_path = 'parsed_output.csv'
with open(fixed_width_file_path, 'r', encoding=spec["FixedWidthEncoding"]) as fw_file, \
     open(csv_file_path, 'w', encoding=spec["DelimitedEncoding"], newline='') as csv_file:
    
    writer = csv.writer(csv_file)
    
    # Write header to CSV
    writer.writerow(spec["ColumnNames"])
    
    # Read and parse the lines
    lines = fw_file.readlines()
    if spec["IncludeHeader"] == "True":
        # Skip the header line in the fixed-width file
        lines = lines[1:]
    
    # Write each parsed line to the CSV
    for line in lines:
        parsed_line = parse_fixed_width_line(line, offsets)
        writer.writerow(parsed_line)